In [5]:
###-----------------
### Import Libraries
###-----------------

import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

In [6]:
###----------------------
### Basic Parameters
###----------------------

inpDir = '../input'
subDir = 'flower_photos'
IMG_HEIGHT = 188
IMG_WIDTH = 188
BATCH_SIZE = 64
EPOCHS = 40
ALPHA = 0.0005
RANDOM_STATE = 24
torch.manual_seed(RANDOM_STATE)

In [7]:
###-----------------
### Data Augmentation
###-----------------
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(IMG_HEIGHT),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_test = transforms.Compose([
    transforms.Resize(IMG_HEIGHT),
    transforms.CenterCrop(IMG_HEIGHT),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [8]:
data_dir = os.path.join(inpDir, subDir)
dataset = datasets.ImageFolder(data_dir, transform=transform_train)

In [9]:
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size
indices = torch.randperm(dataset_size).tolist()

In [10]:
train_dataset = torch.utils.data.Subset(dataset, indices[:train_size])
test_dataset = torch.utils.data.Subset(datasets.ImageFolder(data_dir, transform=transform_test), indices[train_size:])

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [12]:
class_names = train_dataset.dataset.classes
num_classes = len(class_names)

In [13]:
###-----------------
### Model Definition
###-----------------
class ImprovedModel(nn.Module):
    def __init__(self):
        super(ImprovedModel, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
            nn.Dropout(0.4),
            
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2),
            nn.Dropout(0.4)
        )
        self.fc_block = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * (IMG_HEIGHT // 16) * (IMG_WIDTH // 16), 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc_block(x)
        return x

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImprovedModel().to(device)

In [15]:
###-----------------
### Loss, Optimizer, and Scheduler
###-----------------
loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=ALPHA, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

###-----------------
### Training and Evaluation
###-----------------
for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct = 0.0, 0
    for images, labels in tqdm(train_dataloader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_correct += (outputs.argmax(dim=1) == labels).sum().item()

    train_loss /= len(train_dataloader)
    train_acc = train_correct / len(train_dataset)

    model.eval()
    test_loss, test_correct = 0.0, 0
    with torch.no_grad():
        for images, labels in test_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            test_loss += loss.item()
            test_correct += (outputs.argmax(dim=1) == labels).sum().item()

    test_loss /= len(test_dataloader)
    test_acc = test_correct / len(test_dataset)

    scheduler.step(test_loss)

    print(f"Epoch {epoch + 1}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
          f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

/home/duasp/Dec2024/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
100%|██████████| 46/46 [00:15<00:00,  2.96it/s]


Epoch 1/40 | Train Loss: 5.6726 | Train Acc: 0.3324 | Test Loss: 2.0436 | Test Acc: 0.4632


100%|██████████| 46/46 [00:14<00:00,  3.21it/s]


Epoch 2/40 | Train Loss: 1.8776 | Train Acc: 0.4128 | Test Loss: 1.3163 | Test Acc: 0.4673


100%|██████████| 46/46 [00:14<00:00,  3.18it/s]


...

Epoch 38/40 | Train Loss: 0.7653 | Train Acc: 0.7050 | Test Loss: 0.7868 | Test Acc: 0.7466


100%|██████████| 46/46 [00:15<00:00,  3.04it/s]


Epoch 39/40 | Train Loss: 0.7495 | Train Acc: 0.7122 | Test Loss: 0.7416 | Test Acc: 0.7561


100%|██████████| 46/46 [00:15<00:00,  3.03it/s]


Epoch 40/40 | Train Loss: 0.7501 | Train Acc: 0.7119 | Test Loss: 0.7088 | Test Acc: 0.7561
